# Shaper Data Analysis

***
***

## TO DO LIST

- [x] Associare un errore alle misure dirette
- [ ] Calcolare le stime teoriche delle quantità rilevanti
- [ ] Controllare che l'apparato sperimentale risponda secondo le aspettative
- [ ] Confronto la forma d'onda di Arduino con una simulazione Spice
- [ ] Studiare la risposta in frequenza
- [ ] Stimare la banda passante del filtro
- [ ] Confronto con la simulazione Spice
- [ ] Confrontare l'undershoot con Spice
- [ ] Confronto la risposta con Rpz con Spice

## Import modules
***

In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import seaborn as sns
import numpy as np
from scipy import stats
from scipy.optimize import curve_fit
from jupyterthemes import jtplot
jtplot.reset()
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
import importlib
import Shaper_Analysis_Code as SA

In [10]:
importlib.reload(SA)

<module 'Shaper_Analysis_Code' from 'd:\\GitHub\\physics_laboratory_2020_2021\\CATENA ELETTRONICA\\Python\\Shaper_Analysis_Code.py'>

## Misure dirette delle componenti circuitali

***

In tabella sono riportate le misure dirette delle resistenze e della capacità utilizzate per assemblare il circuito

In [8]:
componenti = SA.misure_dirette()
componenti

,Valore,Errore,FS
R1,100.99 kΩ,0.06 kΩ,1000 kΩ
R2,99.93 kΩ,0.06 kΩ,1000 kΩ
C1,157 pF,9 pF,1000 pF
C2,159 pF,9 pF,1000 pF


## Stime Teoriche 

***

Calcolo ora le stime teoriche delle quantità più rilevanti

In [13]:
SA.get_tau_sh_th()

τ_sh_th1 =  15.86 +/- 0.91  μs
τ_sh_th2 =  15.89 +/- 0.90  μs


In [22]:
SA.get_tau_sh_mean()

τ_sh_th =  15.87 +/- 0.64  μs


In [21]:
importlib.reload(SA)

<module 'Shaper_Analysis_Code' from 'd:\\GitHub\\physics_laboratory_2020_2021\\CATENA ELETTRONICA\\Python\\Shaper_Analysis_Code.py'>